# Download data



In [4]:
!pip install -q kaggle
from google.colab import files
files.upload()


!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
# !kaggle datasets download -d sanidhyak/human-face-emotions
# !kaggle datasets download -d sanidhyak/human-face-emotions
!kaggle datasets download -d ananthu017/emotion-detection-fer

Saving kaggle.json to kaggle.json
100% 65.2M/65.2M [00:02<00:00, 32.9MB/s]
100% 65.2M/65.2M [00:02<00:00, 23.7MB/s]


# Extracting files

In [5]:
import zipfile
zip_ref=zipfile.ZipFile('/content/emotion-detection-fer.zip','r')
# zip_ref=zipfile.ZipFile('/content/human-face-emotions.zip','r')
zip_ref.extractall('/content')
zip_ref.close()

# Import Libraries

In [6]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from google.colab.patches import cv2_imshow
import tensorflow as tf
import cv2
# runopts = tf.RunOptions(report_tensor_allocations_upon_oom = True)
import matplotlib.pyplot as plt

# Set the path to your dataset folders
train_path = '/content/data/Happy'
test_path = '/content/data/Sad'
# angry_path = '/content/data/Angry'

In [7]:
train_data_gen = ImageDataGenerator(rescale=1./255)
validation_data_gen = ImageDataGenerator(rescale=1./255)

# Preprocess all test images
train_generator = train_data_gen.flow_from_directory(
        "/content/train",
        target_size=(48, 48),
        batch_size=16,
        color_mode="rgb",
        class_mode='categorical')


test_generator = validation_data_gen.flow_from_directory(
        "/content/test",
        target_size=(48, 48),
        batch_size=16,
        color_mode="rgb",
        class_mode='categorical')

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


# Model Architecture

In [8]:
# create model structure
emotion_model = Sequential()

emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1),name='conv1'))
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu',name='conv2'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2),name='max_pool1'))
emotion_model.add(Dropout(0.25))

emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu',name='con3'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2),name='conv4'))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu',name='conv5'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2),name='conv6'))
emotion_model.add(Dropout(0.25))

emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7, activation='softmax'))

cv2.ocl.setUseOpenCL(False)

emotion_model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.legacy.Adam(lr=0.0001, decay=1e-6), metrics=['accuracy'])

# Train the neural network/model
emotion_model_info = emotion_model.fit(
        x=train_generator,
        epochs=50,validation_data=test_generator
        )

Epoch 1/50


UnimplementedError: ignored

In [4]:

# save model structure in jason file
model_json = emotion_model.to_json()
with open("emotion_model.json", "w") as json_file:
    json_file.write(model_json)

# save trained model weight in .h5 file
emotion_model.save_weights('emotion_model1.h5')

NameError: ignored

In [9]:
# Test Emotion Detection
import cv2
import numpy as np
from keras.models import model_from_json


emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}

# load json and create model
json_file = open('emotion_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
emotion_model = model_from_json(loaded_model_json)

# load weights into new model
emotion_model.load_weights("emotion_model1.h5")
print("Loaded model from disk")

Loaded model from disk


In [12]:
cap = cv2.VideoCapture("new.mp4")

# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # You can use other codecs, e.g., 'MJPG', 'mp4v', 'X264', etc.
out = cv2.VideoWriter('output.mp4', fourcc, fps, (frame_width, frame_height))

while cap.isOpened():
    # Find haar cascade to draw bounding box around face
    ret, frame = cap.read()
    # frame = cv2.resize(frame, (1280, 720),cv2.INTER_AREA)
    if not ret:
        break
    face_detector = cv2.CascadeClassifier('/content/haarcascade-frontalface-default.xml')
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)




    # detect faces available on camera
    num_faces = face_detector.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5)

    # take each face available on the camera and Preprocess it
    for (x, y, w, h) in num_faces:
        cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (0, 255, 0), 4)
        roi_gray_frame = gray_frame[y:y + h, x:x + w]
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame, (48, 48)), -1), 0)

        # predict the emotions
        emotion_prediction = emotion_model.predict(cropped_img)
        maxindex = int(np.argmax(emotion_prediction))
        cv2.putText(frame, emotion_dict[maxindex], (x+5, y-20), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
    out.write(frame)

    # cv2_imshow(frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 27ms/step


In [ ]:
# Evaluate

import numpy as np
from keras.models import model_from_json
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, classification_report,ConfusionMatrixDisplay


emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}

# load json and create model
json_file = open('emotion_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
emotion_model = model_from_json(loaded_model_json)

# load weights into new model
emotion_model.load_weights("emotion_model1.h5")
print("Loaded model from disk")

# Initialize image data generator with rescaling
test_data_gen = ImageDataGenerator(rescale=1./255)

# Preprocess all test images
test_generator = test_data_gen.flow_from_directory(
        'data/test',
        target_size=(48, 48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

# do prediction on test data
predictions = emotion_model.predict_generator(test_generator)

# see predictions
# for result in predictions:
#     max_index = int(np.argmax(result))
#     print(emotion_dict[max_index])

print("-----------------------------------------------------------------")
# confusion matrix
c_matrix = confusion_matrix(test_generator.classes, predictions.argmax(axis=1))
print(c_matrix)
cm_display = ConfusionMatrixDisplay(confusion_matrix=c_matrix, display_labels=emotion_dict)
cm_display.plot(cmap=plt.cm.Blues)
plt.show()

# Classification report
print("-----------------------------------------------------------------")
print(classification_report(test_generator.classes, predictions.argmax(axis=1)))

Loaded model from disk


FileNotFoundError: ignored